In [9]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd
from shapely.geometry import Point

In [2]:
Neighborhood = gpd.read_file("Neighborhood Tabulation Areas (NTA).geojson")

Neighborhood.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005019.048,047,Borough Park,39247.2278309,Brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ..."
1,QN51,52488277.846,081,Murray Hill,33266.9047967,Queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ..."
2,QN27,19726845.6282,081,East Elmhurst,19816.7120736,Queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ..."
3,QN07,22887772.8652,081,Hollis,20976.3355739,Queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ..."
4,MN06,10647077.5264,061,Manhattanville,17040.6854129,Manhattan,1,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ..."


In [23]:
def count_incidents_neighborhood(data, neighb):
 # spatial join and group by to get count of incidents in each poneighbourhood 
    joined = gpd.sjoin(data, neighb, op="within")
    grouped = joined.groupby('ntaname').size()
    df = grouped.to_frame().reset_index()
    df.columns = ['ntaname', 'count']
    merged = Neighborhood.merge(df, on='ntaname', how='outer')
    merged['count'].fillna(0,inplace=True)
    merged['count'] = merged['count'].astype(int)
    return merged

In [6]:
trips = gpd.read_file("train.csv")

trips.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,geometry
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455,None
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663,None
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124,None
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429,None
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625,40.793209075927734,-73.972923278808594,40.782520294189453,N,435,None


In [14]:
pick_ups_frame = { 'id': trips.id, 'lon': trips.pickup_longitude, 'lat': trips.pickup_latitude } 

pick_ups_df = pd.DataFrame(pick_ups_frame) 


In [19]:
longitude = pd.to_numeric(pick_ups_df.lon)
latitude = pd.to_numeric(pick_ups_df.lat)
geometry = [Point(xy) for xy in zip(longitude, latitude)]
crs = {'init': 'epsg:4326'}
pick_ups_gdf = gpd.GeoDataFrame(pick_ups_df, crs=crs, geometry=geometry)

c:\users\vibodha vimarshana\opencv\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
pick_ups_gdf.head()

,id,lon,lat,geometry
0,id2875421,-73.982154846191406,40.767936706542969,POINT (-73.98215 40.76794)
1,id2377394,-73.980415344238281,40.738563537597656,POINT (-73.98042 40.73856)
2,id3858529,-73.979026794433594,40.763938903808594,POINT (-73.97903 40.76394)
3,id3504673,-74.010040283203125,40.719970703125,POINT (-74.01004 40.71997)
4,id2181028,-73.973052978515625,40.793209075927734,POINT (-73.97305 40.79321)


In [24]:
merged_gdf = count_incidents_neighborhood(pick_ups_gdf, Neighborhood)

c:\users\vibodha vimarshana\opencv\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(+init=epsg:4326 +type=crs != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [25]:
merged_gdf.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry,count
0,BK88,54005019.048,047,Borough Park,39247.2278309,Brooklyn,3,"MULTIPOLYGON (((-73.97605 40.63128, -73.97717 ...",28
1,QN51,52488277.846,081,Murray Hill,33266.9047967,Queens,4,"MULTIPOLYGON (((-73.80379 40.77561, -73.80099 ...",11
2,QN27,19726845.6282,081,East Elmhurst,19816.7120736,Queens,4,"MULTIPOLYGON (((-73.86110 40.76366, -73.85993 ...",217
3,QN07,22887772.8652,081,Hollis,20976.3355739,Queens,4,"MULTIPOLYGON (((-73.75726 40.71814, -73.75589 ...",37
4,MN06,10647077.5264,061,Manhattanville,17040.6854129,Manhattan,1,"MULTIPOLYGON (((-73.94608 40.82126, -73.94640 ...",1473


In [26]:
map2 = KeplerGl(height=600, width=800)
# Add data to Kepler
map2.add_data(data=merged_gdf, name="NeighborhoodTrips")
map2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'NeighborhoodTrips': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…